In [6]:
MXE_ENDPOINT="https://mxe-vcluster1.kroto011.rnd.gic.ericsson.se"
MXE_DEPLOYER_ENDPOINT="https://mxe-deployer-vcluster1.kroto011.rnd.gic.ericsson.se"
OAUTH_ENDPOINT="https://oauth.mxe-vcluster1.kroto011.rnd.gic.ericsson.se"
ARGOCD_ENDPOINT="https://argocd-vcluster1.kroto011.rnd.gic.ericsson.se"
GITEA_ENDPOINT="https://gitea.mxe-vcluster1.kroto011.rnd.gic.ericsson.se"
INTERNAL_GITOPS_REPO=f"{GITEA_ENDPOINT}/mxe/mxe-gitops.git"
ARGOCD_ADMIN_USERNAME="argocd-admin"
ARGOCD_ADMIN_PASSWORD="admin-pass"

In [7]:
import os, json, subprocess, pathlib

In [8]:
def run_command(command):
    proc=subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, err = proc.communicate()
    if proc.returncode == 0:
        return out.decode("utf-8")
    else: 
        print(f"Subprocess failed")
        print(f'stdout: {out.decode("utf-8")}')
        print(f'stderr: {err.decode("utf-8")}')
        raise Exception(f"Subprocess failed with return code {proc.returncode}")

## /v1/session

In [9]:
def get_token():
    JSON_PAYLOAD={"ssoHost": OAUTH_ENDPOINT, "ssoMode": True, "username": ARGOCD_ADMIN_USERNAME, "password": ARGOCD_ADMIN_PASSWORD}
    JSON_STR=json.dumps(JSON_PAYLOAD)
    CURL_CMD=f"curl --request POST --url {MXE_DEPLOYER_ENDPOINT}/v1/session -H'Content-Type: application/json' -d '{JSON_STR}'"

    print(f"Get token curl command: {CURL_CMD}")

    output=run_command(CURL_CMD)
    print(f"Output: {output}")
    TOKEN=json.loads(output)["Token"]
    return TOKEN

In [10]:
TOKEN=get_token()
print(TOKEN)

Get token curl command: curl --request POST --url https://mxe-deployer-vcluster1.kroto011.rnd.gic.ericsson.se/v1/session -H'Content-Type: application/json' -d '{"ssoHost": "https://oauth.mxe-vcluster1.kroto011.rnd.gic.ericsson.se", "ssoMode": true, "username": "argocd-admin", "password": "admin-pass"}'
Output: {
  "Token": "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJEcjVXa2tYSmJfS1U3WlBiS0IySFhrd05SVUxRd0Nrb0dwbG40dkZTMXk4In0.eyJleHAiOjE2NjI5NzQzODEsImlhdCI6MTY2Mjk3NDA4MSwianRpIjoiZTdmYzkwNDUtMzU4Zi00OGJhLWE1NTQtMTdlNzdmOTM2YTEyIiwiaXNzIjoiaHR0cHM6Ly9vYXV0aC5teGUtdmNsdXN0ZXIxLmtyb3RvMDExLnJuZC5naWMuZXJpY3Nzb24uc2UvYXV0aC9yZWFsbXMvYXJnb2NkIiwiYXVkIjoiYWNjb3VudCIsInN1YiI6ImQ1MmI5ZWFkLTk2MWEtNDQyNS04NWNmLTAzNTY4MGQyOTcxMSIsInR5cCI6IkJlYXJlciIsImF6cCI6ImFyZ29jZC1yZXN0LWNsaWVudCIsInNlc3Npb25fc3RhdGUiOiIwOGMyZGQ5Ny1kZjEzLTQzNzMtYjJhOS0zMDgwYjY4ODNkYzciLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vYXJnb2NkLXZjbHVzdGVyMS5rcm90bzAxMS5ybmQuZ2ljLmVyaWNzc29uLnNlIl0sInJlYWxtX2FjY2Vzcy

## /v1/package

Make sure https://gitlab.internal.ericsson.com/gaia-mass/mee/deployer-example-apps.git repo is added to ARGOCD_ENDPOINT before proceeding

Verify that this repo is shown in the Settings > Repository page 

<img src="Gitlab_repository.png">

#### Deploy Package from git

In [11]:
TOKEN=get_token()

payload={
    "appName": "sample-nginx-app",
    "packageSource": {
        "repoURL": "https://gitlab.internal.ericsson.com/gaia-mass/mee/deployer-example-apps.git",
        "path": "nginx-sample-yaml",
        "targetRevision": "main"
    },
    "packageDestination": {
        "server" : "https://kubernetes.default.svc",
        "namespace": "mxe"
    },
    "labels": [ "issuedBy=deployer-notebook"],
    "syncPolicy":{
        "automated":{
            "prune": False,
            "selfHeal": False,
            "allowEmpty" : False
        }
    }
}

payload_str=json.dumps(payload)

CURL_CMD=f"""curl --request POST --url {MXE_DEPLOYER_ENDPOINT}/v1/package -H"Authorization: Bearer {TOKEN}" --form 'options={payload_str}'"""

print(f"\n\nDeploy from GIT Curl command: {CURL_CMD}")

output=run_command(CURL_CMD)

print(f"\n\nOutput: {output}")



Get token curl command: curl --request POST --url https://mxe-deployer-vcluster1.kroto011.rnd.gic.ericsson.se/v1/session -H'Content-Type: application/json' -d '{"ssoHost": "https://oauth.mxe-vcluster1.kroto011.rnd.gic.ericsson.se", "ssoMode": true, "username": "argocd-admin", "password": "admin-pass"}'
Output: {
  "Token": "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJEcjVXa2tYSmJfS1U3WlBiS0IySFhrd05SVUxRd0Nrb0dwbG40dkZTMXk4In0.eyJleHAiOjE2NjI5NzQ0MTgsImlhdCI6MTY2Mjk3NDExOCwianRpIjoiY2NmMmI2YTEtMTI4Ni00NGFjLWJlNzYtZGFlOTFkYjcyZjVlIiwiaXNzIjoiaHR0cHM6Ly9vYXV0aC5teGUtdmNsdXN0ZXIxLmtyb3RvMDExLnJuZC5naWMuZXJpY3Nzb24uc2UvYXV0aC9yZWFsbXMvYXJnb2NkIiwiYXVkIjoiYWNjb3VudCIsInN1YiI6ImQ1MmI5ZWFkLTk2MWEtNDQyNS04NWNmLTAzNTY4MGQyOTcxMSIsInR5cCI6IkJlYXJlciIsImF6cCI6ImFyZ29jZC1yZXN0LWNsaWVudCIsInNlc3Npb25fc3RhdGUiOiIwYWNlOGZmYS00N2I5LTQ1NjItOTQ1OS0xNDEwMGZmOTJhNGEiLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vYXJnb2NkLXZjbHVzdGVyMS5rcm90bzAxMS5ybmQuZ2ljLmVyaWNzc29uLnNlIl0sInJlYWxtX2FjY2Vzcy

### Verification

Check ArgoCD applications page, open sample-nginx-app application; it should show synced/healthy

<img src="Successful_deployment_from_git.png" width="800">

#### Verification from k8s

In [12]:
%%bash 

kubectl get application sample-nginx-app -n mxe
kubectl get deployment -l argocd.argoproj.io/instance=sample-nginx-app -n mxe

NAME               SYNC STATUS   HEALTH STATUS
sample-nginx-app   Synced        Healthy
NAME          READY   UP-TO-DATE   AVAILABLE   AGE
nginx-app-1   3/3     3            3           127m


## Deploy Tar file from local fs

Make tar files using sample helm chart helm-examples

In [15]:
%%bash 

[[ -d helm-examples ]] && rm -rf helm-examples
git clone https://github.com/helm/examples.git helm-examples
sed -i 's#repository: nginx#repository: armdockerhub.rnd.ericsson.se/nginx#g' helm-examples/charts/hello-world/values.yaml
tar -czvf helm-examples.tar.gz -C helm-examples/charts/hello-world .
# Change Chartname for tgz
sed -i 's#name: hello-world#name: hello-world-2#g' helm-examples/charts/hello-world/Chart.yaml
tar -czvf helm-examples.tgz -C helm-examples/charts/hello-world .

# Change Chartname for zip
sed -i 's#name: hello-world-2#name: hello-world-3#g' helm-examples/charts/hello-world/Chart.yaml
cd helm-examples/charts/hello-world/ && zip -r $OLDPWD/helm-examples.zip . && cd -

rm -rf archives && mkdir -p archives
mv helm-examples.tar.gz helm-examples.tgz helm-examples.zip archives/

ls archives/

Cloning into 'helm-examples'...


./
./values.yaml
./README.md
./Chart.yaml
./templates/
./templates/serviceaccount.yaml
./templates/deployment.yaml
./templates/_helpers.tpl
./templates/NOTES.txt
./templates/service.yaml
./.helmignore
./
./values.yaml
./README.md
./Chart.yaml
./templates/
./templates/serviceaccount.yaml
./templates/deployment.yaml
./templates/_helpers.tpl
./templates/NOTES.txt
./templates/service.yaml
./.helmignore
  adding: values.yaml (deflated 42%)
  adding: README.md (stored 0%)
  adding: Chart.yaml (deflated 55%)
  adding: templates/ (stored 0%)
  adding: templates/serviceaccount.yaml (deflated 48%)
  adding: templates/deployment.yaml (deflated 64%)
  adding: templates/_helpers.tpl (deflated 69%)
  adding: templates/NOTES.txt (deflated 61%)
  adding: templates/service.yaml (deflated 50%)
  adding: .helmignore (deflated 33%)
/mnt/disk0/enxxram/home/enxxram/workspace/mee/2.4/mxe-core-latest/mxe-deployer/mxe-deployer-service/notebooks
helm-examples.tar.gz
helm-examples.tgz
helm-examples.zip


### Deploy tar file

In [19]:
TOKEN=get_token()

payload={
    "appName": "sample-hello-world-app1",
    "packageSource": {
        "repoURL": INTERNAL_GITOPS_REPO,
        "path": "sample_apps/app1/",
        "targetRevision": "master"
    },
    "packageDestination": {
        "server" : "https://kubernetes.default.svc",
        "namespace": "mxe"
    },
    "labels": [ "issuedBy=deployer-notebook"],
    "syncPolicy":{
        "automated":{
            "prune": False,
            "selfHeal": False,
            "allowEmpty" : False
        }
    }
}
payload_str=json.dumps(payload)

ARCHIVE_PATH=pathlib.Path("archives/helm-examples.tar.gz").absolute()

CURL_CMD=f"""curl --request POST --url {MXE_DEPLOYER_ENDPOINT}/v1/package -H"Authorization: Bearer {TOKEN}" --form archive=@{ARCHIVE_PATH} --form 'options={payload_str}'"""

print(f"\n\nDeploy tar.gz archive Curl command: {CURL_CMD}")

output=run_command(CURL_CMD)

print(f"\n\nDeploy tar.gz archive Output: {output}", flush=True)

Get token curl command: curl --request POST --url https://mxe-deployer-vcluster1.kroto011.rnd.gic.ericsson.se/v1/session -H'Content-Type: application/json' -d '{"ssoHost": "https://oauth.mxe-vcluster1.kroto011.rnd.gic.ericsson.se", "ssoMode": true, "username": "argocd-admin", "password": "admin-pass"}'
Output: {
  "Token": "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJEcjVXa2tYSmJfS1U3WlBiS0IySFhrd05SVUxRd0Nrb0dwbG40dkZTMXk4In0.eyJleHAiOjE2NjI5NzYxNDgsImlhdCI6MTY2Mjk3NTg0OCwianRpIjoiYzMyMzJiY2YtOWZhOC00ZDY3LTlkOGUtN2YyN2U0OTBjYjg2IiwiaXNzIjoiaHR0cHM6Ly9vYXV0aC5teGUtdmNsdXN0ZXIxLmtyb3RvMDExLnJuZC5naWMuZXJpY3Nzb24uc2UvYXV0aC9yZWFsbXMvYXJnb2NkIiwiYXVkIjoiYWNjb3VudCIsInN1YiI6ImQ1MmI5ZWFkLTk2MWEtNDQyNS04NWNmLTAzNTY4MGQyOTcxMSIsInR5cCI6IkJlYXJlciIsImF6cCI6ImFyZ29jZC1yZXN0LWNsaWVudCIsInNlc3Npb25fc3RhdGUiOiI5M2E0ZGQ1OC0wYWQwLTQyNjEtYmI3Yi1hN2FiMDRkMjYzMTUiLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vYXJnb2NkLXZjbHVzdGVyMS5rcm90bzAxMS5ybmQuZ2ljLmVyaWNzc29uLnNlIl0sInJlYWxtX2FjY2Vzcy

### Patch app

In [20]:
TOKEN=get_token()

payload={
    "name": "sample-hello-world-app1",
}
payload_str=json.dumps(payload)

ARCHIVE_PATH=pathlib.Path("archives/helm-examples.tgz").absolute()

CURL_CMD=f"""curl --request PATCH --url {MXE_DEPLOYER_ENDPOINT}/v1/package -H"Authorization: Bearer {TOKEN}" --form archive=@{ARCHIVE_PATH} --form 'appSelector={payload_str}'"""

print(f"\n\Patch app using archive Curl command: {CURL_CMD}")

output=run_command(CURL_CMD)

print(f"\n\nPatch app using archive Output: {output}", flush=True)

Get token curl command: curl --request POST --url https://mxe-deployer-vcluster1.kroto011.rnd.gic.ericsson.se/v1/session -H'Content-Type: application/json' -d '{"ssoHost": "https://oauth.mxe-vcluster1.kroto011.rnd.gic.ericsson.se", "ssoMode": true, "username": "argocd-admin", "password": "admin-pass"}'
Output: {
  "Token": "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJEcjVXa2tYSmJfS1U3WlBiS0IySFhrd05SVUxRd0Nrb0dwbG40dkZTMXk4In0.eyJleHAiOjE2NjI5NzYxOTcsImlhdCI6MTY2Mjk3NTg5NywianRpIjoiYWMyMDJiMmEtMDhmMi00ZDcwLWIxNWMtZGQyYTRlZjk5Yjk1IiwiaXNzIjoiaHR0cHM6Ly9vYXV0aC5teGUtdmNsdXN0ZXIxLmtyb3RvMDExLnJuZC5naWMuZXJpY3Nzb24uc2UvYXV0aC9yZWFsbXMvYXJnb2NkIiwiYXVkIjoiYWNjb3VudCIsInN1YiI6ImQ1MmI5ZWFkLTk2MWEtNDQyNS04NWNmLTAzNTY4MGQyOTcxMSIsInR5cCI6IkJlYXJlciIsImF6cCI6ImFyZ29jZC1yZXN0LWNsaWVudCIsInNlc3Npb25fc3RhdGUiOiIzNzhhZWVlYi1mZWRiLTRiZGEtYWMyZC1iOTJmOGE0ODJlMTUiLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vYXJnb2NkLXZjbHVzdGVyMS5rcm90bzAxMS5ybmQuZ2ljLmVyaWNzc29uLnNlIl0sInJlYWxtX2FjY2Vzcy

### Delete app

In [13]:
def del_app(app_to_delete):
    TOKEN=get_token()
    headers=f"Authorization: Bearer {TOKEN}"

    CURL_CMD=f'curl --request DELETE --url {MXE_DEPLOYER_ENDPOINT}/v1/package?name={app_to_delete} -H"Authorization: Bearer {TOKEN}"'

    print(f"\n\nDelete app Curl command: {CURL_CMD}")

    output=run_command(CURL_CMD)
    print(f"\n\nDelete app Output: {output}")

In [25]:
del_app(app_to_delete="sample-hello-world-app2")

Get token curl command: curl --request POST --url https://mxe-deployer-vcluster1.kroto011.rnd.gic.ericsson.se/v1/session -H'Content-Type: application/json' -d '{"ssoHost": "https://oauth.mxe-vcluster1.kroto011.rnd.gic.ericsson.se", "ssoMode": true, "username": "argocd-admin", "password": "admin-pass"}'
Output: {
  "Token": "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJEcjVXa2tYSmJfS1U3WlBiS0IySFhrd05SVUxRd0Nrb0dwbG40dkZTMXk4In0.eyJleHAiOjE2NjI5Nzk2NTQsImlhdCI6MTY2Mjk3OTM1NCwianRpIjoiMjAwOTllNjUtZjlhNS00MDJhLWE3NDAtMmM3NmNjOGU0YThkIiwiaXNzIjoiaHR0cHM6Ly9vYXV0aC5teGUtdmNsdXN0ZXIxLmtyb3RvMDExLnJuZC5naWMuZXJpY3Nzb24uc2UvYXV0aC9yZWFsbXMvYXJnb2NkIiwiYXVkIjoiYWNjb3VudCIsInN1YiI6ImQ1MmI5ZWFkLTk2MWEtNDQyNS04NWNmLTAzNTY4MGQyOTcxMSIsInR5cCI6IkJlYXJlciIsImF6cCI6ImFyZ29jZC1yZXN0LWNsaWVudCIsInNlc3Npb25fc3RhdGUiOiJhZjM0OTNjZC1hZDI1LTQwYmYtYWM3ZS05MmQ3NjIwNjdjZTYiLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vYXJnb2NkLXZjbHVzdGVyMS5rcm90bzAxMS5ybmQuZ2ljLmVyaWNzc29uLnNlIl0sInJlYWxtX2FjY2Vzcy

## Deploy zip archive

In [22]:
TOKEN=get_token()

payload={
    "appName": "sample-hello-world-app2",
    "packageSource": {
        "repoURL": INTERNAL_GITOPS_REPO,
        "path": "sample_apps/app2/",
        "targetRevision": "master"
    },
    "packageDestination": {
        "server" : "https://kubernetes.default.svc",
        "namespace": "mxe"
    },
    "labels": [ "issuedBy=deployer-notebook"],
    "syncPolicy":{
        "automated":{
            "prune": False,
            "selfHeal": False,
            "allowEmpty" : False
        }
    }
}
payload_str=json.dumps(payload)

ARCHIVE_PATH=pathlib.Path("archives/helm-examples.zip").absolute()

CURL_CMD=f"""curl --request POST --url {MXE_DEPLOYER_ENDPOINT}/v1/package -H"Authorization: Bearer {TOKEN}" --form archive=@{ARCHIVE_PATH} --form 'options={payload_str}'"""

print(f"\n\nDeploy zip archive Curl command: {CURL_CMD}")

output=run_command(CURL_CMD)

print(f"\n\nDeploy zip archive Output: {output}", flush=True)

Get token curl command: curl --request POST --url https://mxe-deployer-vcluster1.kroto011.rnd.gic.ericsson.se/v1/session -H'Content-Type: application/json' -d '{"ssoHost": "https://oauth.mxe-vcluster1.kroto011.rnd.gic.ericsson.se", "ssoMode": true, "username": "argocd-admin", "password": "admin-pass"}'
Output: {
  "Token": "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJEcjVXa2tYSmJfS1U3WlBiS0IySFhrd05SVUxRd0Nrb0dwbG40dkZTMXk4In0.eyJleHAiOjE2NjI5NzYyMzMsImlhdCI6MTY2Mjk3NTkzMywianRpIjoiNGRjMmI1MGItN2JkYi00ZWI0LWI5ZjQtNDcyY2QwYzVkMmNmIiwiaXNzIjoiaHR0cHM6Ly9vYXV0aC5teGUtdmNsdXN0ZXIxLmtyb3RvMDExLnJuZC5naWMuZXJpY3Nzb24uc2UvYXV0aC9yZWFsbXMvYXJnb2NkIiwiYXVkIjoiYWNjb3VudCIsInN1YiI6ImQ1MmI5ZWFkLTk2MWEtNDQyNS04NWNmLTAzNTY4MGQyOTcxMSIsInR5cCI6IkJlYXJlciIsImF6cCI6ImFyZ29jZC1yZXN0LWNsaWVudCIsInNlc3Npb25fc3RhdGUiOiIwZGJlYjk1YS0zNGQ2LTRkODMtOGFjZS1lM2M3YjU2YTM4NGEiLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vYXJnb2NkLXZjbHVzdGVyMS5rcm90bzAxMS5ybmQuZ2ljLmVyaWNzc29uLnNlIl0sInJlYWxtX2FjY2Vzcy

### Get Applications

In [23]:
TOKEN=get_token()
## Set selector to None to get all apps
#SELECTOR=None
# Set selector to a label to get apps with that label
#SELECTOR="issuedBy=deployer-notebook"
# Set selector name to get app by name
SELECTOR="name=sample-hello-world-app2"

ENDPOINT=f"{MXE_DEPLOYER_ENDPOINT}/v1/package"

if SELECTOR:
    ENDPOINT=f"{ENDPOINT}?{SELECTOR}"

CURL_CMD=f'curl --request GET --url {ENDPOINT} -H"Authorization: Bearer {TOKEN}"'

print(f"\n\nGet apps Curl command: {CURL_CMD}")

output=run_command(CURL_CMD)

print(f"\n\nGet apps Output: {output}")


Get token curl command: curl --request POST --url https://mxe-deployer-vcluster1.kroto011.rnd.gic.ericsson.se/v1/session -H'Content-Type: application/json' -d '{"ssoHost": "https://oauth.mxe-vcluster1.kroto011.rnd.gic.ericsson.se", "ssoMode": true, "username": "argocd-admin", "password": "admin-pass"}'
Output: {
  "Token": "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJEcjVXa2tYSmJfS1U3WlBiS0IySFhrd05SVUxRd0Nrb0dwbG40dkZTMXk4In0.eyJleHAiOjE2NjI5NzYyODYsImlhdCI6MTY2Mjk3NTk4NiwianRpIjoiMzY1ZWYzMWMtMjdhNy00MmIwLThhNGQtMTY1MjhkMDg2MGMzIiwiaXNzIjoiaHR0cHM6Ly9vYXV0aC5teGUtdmNsdXN0ZXIxLmtyb3RvMDExLnJuZC5naWMuZXJpY3Nzb24uc2UvYXV0aC9yZWFsbXMvYXJnb2NkIiwiYXVkIjoiYWNjb3VudCIsInN1YiI6ImQ1MmI5ZWFkLTk2MWEtNDQyNS04NWNmLTAzNTY4MGQyOTcxMSIsInR5cCI6IkJlYXJlciIsImF6cCI6ImFyZ29jZC1yZXN0LWNsaWVudCIsInNlc3Npb25fc3RhdGUiOiIxYzQ3YWM0NC02NThhLTRhYmEtYmU4Yi0zYzYwY2NhYzNlZjgiLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vYXJnb2NkLXZjbHVzdGVyMS5rcm90bzAxMS5ybmQuZ2ljLmVyaWNzc29uLnNlIl0sInJlYWxtX2FjY2Vzcy

### App sync command

In [24]:
TOKEN=get_token()
payload={ "applicationSyncReq": { "name" : "sample-hello-world-app2", "prune": True } }

payload_str=json.dumps(payload)

CURL_CMD=f"""curl --request POST --header "Authorization: Bearer {TOKEN}" --url {MXE_DEPLOYER_ENDPOINT}/v1/package/sync --data '{payload_str}'"""

print(f"\n\nSync app Curl command: {CURL_CMD}")

output=run_command(CURL_CMD)

print(f"\n\nSync app Output: {output}", flush=True)


Get token curl command: curl --request POST --url https://mxe-deployer-vcluster1.kroto011.rnd.gic.ericsson.se/v1/session -H'Content-Type: application/json' -d '{"ssoHost": "https://oauth.mxe-vcluster1.kroto011.rnd.gic.ericsson.se", "ssoMode": true, "username": "argocd-admin", "password": "admin-pass"}'
Output: {
  "Token": "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJEcjVXa2tYSmJfS1U3WlBiS0IySFhrd05SVUxRd0Nrb0dwbG40dkZTMXk4In0.eyJleHAiOjE2NjI5NzYzMDUsImlhdCI6MTY2Mjk3NjAwNSwianRpIjoiM2NiYjk3N2EtN2VjYS00MGZhLWE2YmQtMmM5NDBlYWM1MGEzIiwiaXNzIjoiaHR0cHM6Ly9vYXV0aC5teGUtdmNsdXN0ZXIxLmtyb3RvMDExLnJuZC5naWMuZXJpY3Nzb24uc2UvYXV0aC9yZWFsbXMvYXJnb2NkIiwiYXVkIjoiYWNjb3VudCIsInN1YiI6ImQ1MmI5ZWFkLTk2MWEtNDQyNS04NWNmLTAzNTY4MGQyOTcxMSIsInR5cCI6IkJlYXJlciIsImF6cCI6ImFyZ29jZC1yZXN0LWNsaWVudCIsInNlc3Npb25fc3RhdGUiOiJlYmY4OTRjYS1jODMwLTQ4YzctODc5Mi03MWVkMGYyODRlZTciLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vYXJnb2NkLXZjbHVzdGVyMS5rcm90bzAxMS5ybmQuZ2ljLmVyaWNzc29uLnNlIl0sInJlYWxtX2FjY2Vzcy